In [33]:
# chavez & coombs 
# cognos_data_management_13976(v3).ipynb
# 2020-08-13
# this script makes cognos-output data compatible with chavez-created analytic processes

Steps:
1. Download the deltek data as an Excel file
2. Save 'AP2 Regular & OT Hours for Adv by CCC_#505ap.xlsx' as a CSV file
3. Run the code below

In [34]:
# to determine conda version or conda and python versions, from a cmd window at the ">" prompt:
# > conda -V
#   conda 4.2.9
# > python --version
#   Python 3.5.2 :: Anaconda 4.2.0 (64-bit)  

In [35]:
# import modules
import pandas
import os
import sys
import gc
# import numpy
# import matplotlib
# import pylab

In [36]:
# current working directory
print(os.getcwd())

\\ain2\dfsroot\userdata48\willitc0\Data


"read_csv() vs read_excel() in pandas: When to use which and why" by Ashwin A. Vardhan

Have you ever wondered if that excel file that you have, can be made to read faster instead of sitting idle for 10 minutes while your code scans through it? I’ll try to answer it here with a personal experience of mine.

So, I was performing some operations on an excel file using pandas, whose dimensions were 509579 x 240, and it had a size of 295 MB. However, reading that file took about 528 seconds (average over 10 iterations), whereas, on converting it to csv and reading it (using pandas) took just 13 seconds (again, average over 10 iterations), which is an improvement of about 40 times. As you can observe, the file was too huge, and read_excel is just slower in performance. This has been mentioned on stackoverflow too.

So, when will you actually need to convert an excel file to csv before processing it? Usually, if your excel file is small (~100,000 rows, ~50 columns), there won’t be much of a performance issue, unless, you need to run that process very frequently, because, that small delay may get compunded and bite you! But, if your excel file is massive, and you need to process it frequently (or not), it’s better to first convert the excel into a csv, and voila! see the magic happen.

In [37]:
# read data
# df = pd.read_csv (r'Path where the CSV file is stored\File name.csv')
data = pandas.read_csv (r'U:\AP2 Regular & OT Hours for Adv by CCC_#505ap.csv', low_memory=False)

Pandas has the following data types:
    object
    int64
    float64
    bool
    datetime64
    timedelta[ns]
    category

In [38]:
# check metadata
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221876 entries, 0 to 221875
Data columns (total 23 columns):
User                 221876 non-null object
User Id              221876 non-null object
Work Date            221876 non-null object
Month                221876 non-null object
Description          221876 non-null object
REF ID               221876 non-null int64
CLIN                 221876 non-null object
WBS                  38596 non-null object
TASK                 221876 non-null object
TPID                 221876 non-null object
ProjNo               221876 non-null object
WB2                  221876 non-null object
Project Id           221876 non-null object
Pay Type             221876 non-null object
Corporation          221876 non-null object
Company Name         221876 non-null object
Approver             221876 non-null object
Status               221876 non-null object
Straight             221876 non-null float64
Overtime             221876 non-null float64
Total    

In [39]:
# trim data
df = pandas.DataFrame(data, columns= ['User','Work Date', 'Description', 'TPID', 'Straight', 'Overtime', 'Total'])
df.head(2)

,User,Work Date,Description,TPID,Straight,Overtime,Total
0,"SCHNEIDER, LAURA E","Jul 14, 2020",Standby Only - COVID-19,13976.00.01.0111.999,0.00,0.00,0.00
1,"SCHNEIDER, LAURA E","Jul 17, 2020",Standby Only - COVID-19,13976.00.01.0111.999,0.00,0.00,0.00


In [40]:
# check metadata
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221876 entries, 0 to 221875
Data columns (total 7 columns):
User           221876 non-null object
Work Date      221876 non-null object
Description    221876 non-null object
TPID           221876 non-null object
Straight       221876 non-null float64
Overtime       221876 non-null float64
Total          221876 non-null float64
dtypes: float64(3), object(4)
memory usage: 11.8+ MB


In [41]:
# check dataframe dimensions (rows x columns)
df.shape

(221876, 7)

In [42]:
# format data
# round floating numbers to two decimal places in python pandas 
pandas.options.display.float_format = '{:.2f}'.format

In [43]:
# check data types
df.dtypes

User            object
Work Date       object
Description     object
TPID            object
Straight       float64
Overtime       float64
Total          float64
dtype: object

In [44]:
# add clin placeholder column
df.insert(3, 'CLIN', '')
df.head(3)

,User,Work Date,Description,CLIN,TPID,Straight,Overtime,Total
0,"SCHNEIDER, LAURA E","Jul 14, 2020",Standby Only - COVID-19,,13976.00.01.0111.999,0.00,0.00,0.00
1,"SCHNEIDER, LAURA E","Jul 17, 2020",Standby Only - COVID-19,,13976.00.01.0111.999,0.00,0.00,0.00
2,"SCHNEIDER, LAURA E","Jul 20, 2020",Standby Only - COVID-19,,13976.00.01.0111.999,8.00,0.00,8.00


In [45]:
# add wbs placeholder column
df.insert(4, 'WBS', '')
df.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total
0,"SCHNEIDER, LAURA E","Jul 14, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00
1,"SCHNEIDER, LAURA E","Jul 17, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00
2,"SCHNEIDER, LAURA E","Jul 20, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,8.00,0.00,8.00


In [46]:
# add SR placeholder column
df.insert(9, 'SR_temp', '')
df.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp
0,"SCHNEIDER, LAURA E","Jul 14, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00,
1,"SCHNEIDER, LAURA E","Jul 17, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00,
2,"SCHNEIDER, LAURA E","Jul 20, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,8.00,0.00,8.00,


In [47]:
# add option year column
df.insert(10, 'OptYr', 'OY.')
df.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr
0,"SCHNEIDER, LAURA E","Jul 14, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00,,OY.
1,"SCHNEIDER, LAURA E","Jul 17, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00,,OY.
2,"SCHNEIDER, LAURA E","Jul 20, 2020",Standby Only - COVID-19,,,13976.00.01.0111.999,8.00,0.00,8.00,,OY.


In [48]:
# check metadata
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221876 entries, 0 to 221875
Data columns (total 11 columns):
User           221876 non-null object
Work Date      221876 non-null object
Description    221876 non-null object
CLIN           221876 non-null object
WBS            221876 non-null object
TPID           221876 non-null object
Straight       221876 non-null float64
Overtime       221876 non-null float64
Total          221876 non-null float64
SR_temp        221876 non-null object
OptYr          221876 non-null object
dtypes: float64(3), object(8)
memory usage: 18.6+ MB


In [49]:
# column a
df['User'].head(2)

0    SCHNEIDER, LAURA E
1    SCHNEIDER, LAURA E
Name: User, dtype: object

In [50]:
# column b
# modify workdate format
df['Work Date'] = pandas.to_datetime(df['Work Date']).dt.strftime('%Y-%m-%d')
df.head(2)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00,,OY.
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,,,13976.00.01.0111.999,0.00,0.00,0.00,,OY.


In [51]:
# column c
df['Description'].head(2)

0    Standby Only - COVID-19
1    Standby Only - COVID-19
Name: Description, dtype: object

In [52]:
# column d
# get clin from tpid
df['CLIN'] = data.TPID.str.split(".", expand=True,)[3]
df.head(2)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,,13976.00.01.0111.999,0.00,0.00,0.00,,OY.
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,,13976.00.01.0111.999,0.00,0.00,0.00,,OY.


In [53]:
# column e
# get wbs from tpid
df['WBS'] = data.TPID.str.split(".", expand=True,)[4]
df.head(2)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.


In [54]:
# column f
# tpid
df.head(2)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.


In [55]:
# save syntax to modfiy column order
# df = df[['User', 'Work Date', 'Description', 'CLIN', 'WBS', 'TPID', 'Straight', 'Overtime', 'Total', 'SR', 'OptYr']]

In [56]:
# check metadata
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221876 entries, 0 to 221875
Data columns (total 11 columns):
User           221876 non-null object
Work Date      221876 non-null object
Description    221876 non-null object
CLIN           221876 non-null object
WBS            221876 non-null object
TPID           221876 non-null object
Straight       221876 non-null float64
Overtime       221876 non-null float64
Total          221876 non-null float64
SR_temp        221876 non-null object
OptYr          221876 non-null object
dtypes: float64(3), object(8)
memory usage: 18.6+ MB


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221876 entries, 0 to 221875
Data columns (total 23 columns):
User                 221876 non-null object
User Id              221876 non-null object
Work Date            221876 non-null object
Month                221876 non-null object
Description          221876 non-null object
REF ID               221876 non-null int64
CLIN                 221876 non-null object
WBS                  38596 non-null object
TASK                 221876 non-null object
TPID                 221876 non-null object
ProjNo               221876 non-null object
WB2                  221876 non-null object
Project Id           221876 non-null object
Pay Type             221876 non-null object
Corporation          221876 non-null object
Company Name         221876 non-null object
Approver             221876 non-null object
Status               221876 non-null object
Straight             221876 non-null float64
Overtime             221876 non-null float64
Total    

In [58]:
# what size are some of the objects?
print('data', sys.getsizeof(data))
print('df', sys.getsizeof(df))
print('Straight', sys.getsizeof(df['Straight']))
print('TPID', sys.getsizeof(df['TPID']))

data 284431322
df 123370935
Straight 1775112
TPID 17393324


In [59]:
# delete original dataframe to free up memory
del[data]

In [60]:
# save delete datafrome column syntax
# df = df.drop('Company Name', 1)

In [61]:
gc.collect()

68

In [62]:
# add tpid character count column
df['tpid_length'] = df['TPID'].str.len()
df['tpid_length'].head(3)

0    20
1    20
2    20
Name: tpid_length, dtype: int64

In [63]:
# where tpid_length = 28
df['tpid_length'][df['tpid_length'] == 28].head(3)

2316    28
2317    28
2318    28
Name: tpid_length, dtype: int64

In [64]:
df.head(2)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr,tpid_length
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.,20
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.,20


In [65]:
# where tpid_length != 28
df[df['tpid_length'] != 28].head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr,tpid_length
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.,20
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.,20
2,"SCHNEIDER, LAURA E",2020-07-20,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,8.00,0.00,8.00,,OY.,20


In [66]:
# filter on tpid_length
df20 = df[df['tpid_length'] != 28]
df20.shape

(183280, 12)

In [67]:
df20.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr,tpid_length
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.,20
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,OY.,20
2,"SCHNEIDER, LAURA E",2020-07-20,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,8.00,0.00,8.00,,OY.,20


In [68]:
# filter on tpid_length
df28 = df[df['tpid_length'] == 28]
df28.shape

(38596, 12)

In [69]:
del[df]
gc.collect()

472

In [70]:
# SR column
df20.insert(10, 'SR', 'SR-0000000')
df20.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,SR,OptYr,tpid_length
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,SR-0000000,OY.,20
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,,SR-0000000,OY.,20
2,"SCHNEIDER, LAURA E",2020-07-20,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,8.00,0.00,8.00,,SR-0000000,OY.,20


In [71]:
# delete SR_temp column
df20 = df20.drop('SR_temp', 1)
df20.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR,OptYr,tpid_length
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.,20
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.,20
2,"SCHNEIDER, LAURA E",2020-07-20,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,8.00,0.00,8.00,SR-0000000,OY.,20


In [72]:
df28.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr,tpid_length
2316,"TIMMONS, CARL A",2019-08-22,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5522685,8.00,0.00,8.00,,OY.,28
2317,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5364379,8.00,0.00,8.00,,OY.,28
2318,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5262550,8.00,0.00,8.00,,OY.,28


In [73]:
# get sr from tpid
df28['SR_no'] = df28.TPID.str.split(".", expand=True,)[5]
df28.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,OptYr,tpid_length,SR_no
2316,"TIMMONS, CARL A",2019-08-22,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5522685,8.00,0.00,8.00,,OY.,28,5522685
2317,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5364379,8.00,0.00,8.00,,OY.,28,5364379
2318,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5262550,8.00,0.00,8.00,,OY.,28,5262550


In [74]:
df28.insert(10, 'SR_prefix', 'SR-')
df28.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,SR_prefix,OptYr,tpid_length,SR_no
2316,"TIMMONS, CARL A",2019-08-22,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5522685,8.00,0.00,8.00,,SR-,OY.,28,5522685
2317,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5364379,8.00,0.00,8.00,,SR-,OY.,28,5364379
2318,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5262550,8.00,0.00,8.00,,SR-,OY.,28,5262550


In [75]:
df28['SR'] = df28['SR_prefix'] + df28['SR_no']
df28.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_temp,SR_prefix,OptYr,tpid_length,SR_no,SR
2316,"TIMMONS, CARL A",2019-08-22,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5522685,8.00,0.00,8.00,,SR-,OY.,28,5522685,SR-5522685
2317,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5364379,8.00,0.00,8.00,,SR-,OY.,28,5364379,SR-5364379
2318,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5262550,8.00,0.00,8.00,,SR-,OY.,28,5262550,SR-5262550


In [76]:
# delete SR_temp column
df28 = df28.drop('SR_temp', 1)
df28.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR_prefix,OptYr,tpid_length,SR_no,SR
2316,"TIMMONS, CARL A",2019-08-22,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5522685,8.00,0.00,8.00,SR-,OY.,28,5522685,SR-5522685
2317,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5364379,8.00,0.00,8.00,SR-,OY.,28,5364379,SR-5364379
2318,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5262550,8.00,0.00,8.00,SR-,OY.,28,5262550,SR-5262550


In [77]:
# delete SR_prefix column
df28 = df28.drop('SR_prefix', 1)
df28.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,OptYr,tpid_length,SR_no,SR
2316,"TIMMONS, CARL A",2019-08-22,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5522685,8.00,0.00,8.00,OY.,28,5522685,SR-5522685
2317,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5364379,8.00,0.00,8.00,OY.,28,5364379,SR-5364379
2318,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5262550,8.00,0.00,8.00,OY.,28,5262550,SR-5262550


In [78]:
# delete SR_no column
df28 = df28.drop('SR_no', 1)
df28.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,OptYr,tpid_length,SR
2316,"TIMMONS, CARL A",2019-08-22,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5522685,8.00,0.00,8.00,OY.,28,SR-5522685
2317,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5364379,8.00,0.00,8.00,OY.,28,SR-5364379
2318,"TIMMONS, CARL A",2019-09-12,Installation-Fixed Price,0014,000,13976.00.00.0014.000.5262550,8.00,0.00,8.00,OY.,28,SR-5262550


In [79]:
# standardize column order
df20 = df20[['User', 'Work Date', 'Description', 'CLIN', 'WBS', 'TPID', 'Straight', 'Overtime', 'Total', 'SR', 'OptYr', 'tpid_length']]
df28 = df28[['User', 'Work Date', 'Description', 'CLIN', 'WBS', 'TPID', 'Straight', 'Overtime', 'Total', 'SR', 'OptYr', 'tpid_length']]

In [80]:
# concatenate vertical dataframes
df = pandas.concat([df20, df28], axis=0)

In [81]:
df20.shape

(183280, 12)

In [82]:
df28.shape

(38596, 12)

In [83]:
# check columns
df.shape

(221876, 12)

In [84]:
del[df20, df28]

In [85]:
gc.collect()

279

In [86]:
# write data to csv (default is U:\Documents)
df.to_csv('AP2 Regular & OT Hours for Adv by CCC_#505ap-modified.csv')

In [87]:
# check metadata
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221876 entries, 0 to 221834
Data columns (total 12 columns):
User           221876 non-null object
Work Date      221876 non-null object
Description    221876 non-null object
CLIN           221876 non-null object
WBS            221876 non-null object
TPID           221876 non-null object
Straight       221876 non-null float64
Overtime       221876 non-null float64
Total          221876 non-null float64
SR             221876 non-null object
OptYr          221876 non-null object
tpid_length    221876 non-null int64
dtypes: float64(3), int64(1), object(8)
memory usage: 22.0+ MB


In [88]:
# data check
df.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR,OptYr,tpid_length
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.,20
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.,20
2,"SCHNEIDER, LAURA E",2020-07-20,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,8.00,0.00,8.00,SR-0000000,OY.,20


In [89]:
# tip 1
# create dataframe with mixed data types
pandas.util.testing.makeMixedDataFrame()

,A,B,C,D
0,0.00,0.00,foo1,2009-01-01
1,1.00,1.00,foo2,2009-01-02
2,2.00,0.00,foo3,2009-01-05
3,3.00,1.00,foo4,2009-01-06
4,4.00,0.00,foo5,2009-01-07


In [90]:
# tip 2
# insert a new column which is a function of another column
dftest = pandas.util.testing.makeMixedDataFrame()
#print(dftest)
dftest.insert(3, 'A2', dftest['A'] * 2)
dftest

,A,B,C,A2,D
0,0.00,0.00,foo1,0.00,2009-01-01
1,1.00,1.00,foo2,2.00,2009-01-02
2,2.00,0.00,foo3,4.00,2009-01-05
3,3.00,1.00,foo4,6.00,2009-01-06
4,4.00,0.00,foo5,8.00,2009-01-07


In [91]:
# tip 3
# insert a new column with a constant value in the column with index 3 (i.e., 4)
dftest = pandas.util.testing.makeMixedDataFrame()
dftest.insert(3, 'Country', 'USA')
dftest

,A,B,C,Country,D
0,0.00,0.00,foo1,USA,2009-01-01
1,1.00,1.00,foo2,USA,2009-01-02
2,2.00,0.00,foo3,USA,2009-01-05
3,3.00,1.00,foo4,USA,2009-01-06
4,4.00,0.00,foo5,USA,2009-01-07


In [92]:
# tip 4
# change column names
dftest = pandas.util.testing.makeMixedDataFrame()
dftest
dftest.rename(columns={'A':'ID', 'D':'Date'})

,ID,B,C,Date
0,0.00,0.00,foo1,2009-01-01
1,1.00,1.00,foo2,2009-01-02
2,2.00,0.00,foo3,2009-01-05
3,3.00,1.00,foo4,2009-01-06
4,4.00,0.00,foo5,2009-01-07


In [93]:
# tip 5
# duplicate a column
dftest = pandas.util.testing.makeMixedDataFrame()
dftest['E'] = dftest['C']
dftest

,A,B,C,D,E
0,0.00,0.00,foo1,2009-01-01,foo1
1,1.00,1.00,foo2,2009-01-02,foo2
2,2.00,0.00,foo3,2009-01-05,foo3
3,3.00,1.00,foo4,2009-01-06,foo4
4,4.00,0.00,foo5,2009-01-07,foo5


In [94]:
# tip 6
# determine dataframe datatypes
dftest = pandas.util.testing.makeMixedDataFrame()
dataTypeSeries = dftest.dtypes
print(dataTypeSeries)

A           float64
B           float64
C            object
D    datetime64[ns]
dtype: object


In [95]:
# tip 7
# delete a dataframe column

# df = df.drop('column_name', 1)
# where 1 is the axis number (0 for rows and 1 for columns.)

# To delete the column without having to reassign df you can do:
# df.drop('column_name', axis=1, inplace=True)

# To drop by column number instead of by column label, try this to delete, e.g. the 1st, 2nd and 4th columns:
# df = df.drop(df.columns[[0, 1, 3]], axis=1)  
# df.columns is zero-based pd.Index 

# Working with "text" syntax for the columns:
# df.drop(['column_nameA', 'column_nameB'], axis=1, inplace=True)

dftest = pandas.util.testing.makeMixedDataFrame()
print(dftest)
dftest = dftest.drop('C', 1)
print(dftest)

     A    B     C          D
0 0.00 0.00  foo1 2009-01-01
1 1.00 1.00  foo2 2009-01-02
2 2.00 0.00  foo3 2009-01-05
3 3.00 1.00  foo4 2009-01-06
4 4.00 0.00  foo5 2009-01-07
     A    B          D
0 0.00 0.00 2009-01-01
1 1.00 1.00 2009-01-02
2 2.00 0.00 2009-01-05
3 3.00 1.00 2009-01-06
4 4.00 0.00 2009-01-07


In [96]:
# tip 8
# reorder columns (flipflop column a and column d)
dftest = pandas.util.testing.makeMixedDataFrame()
print(dftest)
dftest = dftest[['D','B','C','A']]
print(dftest)

     A    B     C          D
0 0.00 0.00  foo1 2009-01-01
1 1.00 1.00  foo2 2009-01-02
2 2.00 0.00  foo3 2009-01-05
3 3.00 1.00  foo4 2009-01-06
4 4.00 0.00  foo5 2009-01-07
           D    B     C    A
0 2009-01-01 0.00  foo1 0.00
1 2009-01-02 1.00  foo2 1.00
2 2009-01-05 0.00  foo3 2.00
3 2009-01-06 1.00  foo4 3.00
4 2009-01-07 0.00  foo5 4.00


In [97]:
df.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR,OptYr,tpid_length
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.,20
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.,20
2,"SCHNEIDER, LAURA E",2020-07-20,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,8.00,0.00,8.00,SR-0000000,OY.,20


In [98]:
# delete tpid_length column
df = df.drop('tpid_length', 1)
df.head(3)

,User,Work Date,Description,CLIN,WBS,TPID,Straight,Overtime,Total,SR,OptYr
0,"SCHNEIDER, LAURA E",2020-07-14,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.
1,"SCHNEIDER, LAURA E",2020-07-17,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,0.00,0.00,0.00,SR-0000000,OY.
2,"SCHNEIDER, LAURA E",2020-07-20,Standby Only - COVID-19,0111,999,13976.00.01.0111.999,8.00,0.00,8.00,SR-0000000,OY.


In [99]:
df.to_excel (r'U:\AP2 Regular & OT Hours for Adv by CCC_#505ap-modified.xlsx', index = False, header=True)